### DATA COLLECTING

In [1]:
import re
import pandas as pd
import json

from pythainlp.tokenize import THAI2FIT_TOKENIZER

# message converter
from project_module.message_data_converter import MessageDataConverter


In [2]:
json_file = open('./raw_data/wimon_message_1.json')
json_obj = json.load(json_file)
message_list = []

for message in json_obj['messages']:
    try:
        message_content = message['content'].encode('latin_1').decode('utf-8')
        message_list.append(message_content)
        
    except:
        continue

In [3]:
print(message_list)

['ขอบคุณค่ะ', 'โอเคค่ะ', 'ให้อาจารย์เซ็นเรียบร้อยแล้วครับ', 'ได้ค่ะ', 'โอเคครับ ผมขอเอาไปให้อาจารย์เซ็นก่อนครับ', 'ทั้งสองคนเลย', 'มีค่ะ', 'ขอโทษครับ ผมลืมถามเรื่องลายเซ็นต้องมีของอาจารย์ที่ปรึกษากับอาจารย์รายวิชาเลยไหมครับ', 'เคค่ะ', 'ได้เลยครับ', 'ส่งให้พี่วันศุกร์ นะคะ', 'ขอบคุณค่ะ', 'โอเคครับ', 'กรอกใบสมัครด้วยนะ', 'เคค่ะ', 'คุยแล้วครับ', 'อาจารย์คุยกับน้องแล้วใช่ไหม', 'เวลาทำงาน', 'การทำงาน คือ ทำ 6 ชม. ต่อสัปดาห์นะคะ', 'มีตรวจงานบ้าง', 'งาน ก็มี ช่วยดูระหว่างน้องลงแลบ ค่ะ', 'TA 241-151', 'น้องครับ', 'เคค่ะ', 'ยังเลยครับ', 'อาจารย์ เค้าทักมาคุยยัง', 'ขอบคุณค่ะ', 'รับทราบครับ', 'เดียว อ.สมชัย จะติดต่อเราผ่านทางเฟสบุ๊ค นะคะ', 'เคค่ะ', '087-6313043 ครับ', 'พี่ขอเบอร์โทรเราหน่อย', 'ได้ค่ะ', 'เวลาไม่ชนครับ ผมขอทราบลักษณะงานก่อนนะครับ', 'ได้ค่ะ', 'สักครู่นะครับ กำลังเช็คตารางอยู่ครับ', 'ถ้าได้ พี่ให้คุย กับอ.สมชัย ถามลักษณะงานก่อน', 'ช่วงดังกล่าว มีเรียนไหม', 'ได้ไหม', 'ทำงาน วัน จันทร์ เวลา 13.00 - 16.00 และวันพุธ เวลา 13.00 -16.00น.', 'พี่จะขอทาบทามเรา เป็นTA วิชา 241-151 หลักสูตร IA 

## Filter non contain thai character 

In [ ]:
# finding constant values
text_bytes_code = 'ก'.encode('utf-8')
print(text_bytes_code)
print(len(text_bytes_code))

text_int_code = int.from_bytes(text_bytes_code, 'big')
print(text_int_code)

text_int_code += 62

text_bytes_code_last = text_int_code.to_bytes(len(text_bytes_code), 'big')
print(text_bytes_code_last)
text_last = text_bytes_code_last.decode('utf-8')

print(text_last)

In [ ]:
# use constanst values to create map
thai_characters_offset = 14727297
thai_characters_bytes_len = 3
thai_characters = [ (index.to_bytes(thai_characters_bytes_len, 'big')).decode('utf-8') for index in range(thai_characters_offset, thai_characters_offset + 63) ]
print(thai_characters)

In [ ]:
# filtter messages
filter_func = lambda s: any(x in s for x in thai_characters)

message_list_filtered = [line for line in message_list if filter_func(line)]

print(message_list_filtered)

### EXPORT TO CSV

In [ ]:
data = pd.DataFrame({'message': message_list_filtered})
data

In [ ]:
data.to_csv('message_csv.csv', encoding='utf-8-sig')

# Use Case

In [ ]:
message_data_converter = MessageDataConverter()

In [ ]:
message_data_converter.import_json(path='./raw_data/wimon_message_1.json')

In [ ]:
message_data_converter.get_message_list()

In [ ]:
message_data_converter.export_csv(path='./data/message_temp.csv')

In [ ]:
# message_data_converter.merge_csv(csv1_path='./data/message_1.csv', csv2_path= './data/message_2.csv', des_path='./data/message_12.csv')

### Many JSON in folder to CSV

In [ ]:
# init converter
message_data_converter = MessageDataConverter()

In [ ]:
# mypath = './raw_data/'
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
# print(onlyfiles)
# for f in onlyfiles:
#     file_name = f.split('.')[0]
#     message_data_converter.import_json(path = f'./raw_data/{f}')
#     message_data_converter.export_csv(path = f'./data/{file_name}.csv')
#     message_data_converter.clear_message_list()


In [ ]:
#completed many file convert func. in module
message_data_converter.convert_many(json_path= './raw_data/', csv_path= './data/')

### Merged Multiple CSV

In [ ]:
message_data_converter.merge_many_csv(src_path='./test_data/', des_path='./data/')

### Count Words in Dataframe

In [ ]:
data = pd.read_csv("./data/message_temp.csv")
data = data[['message', 'category']]

word_list = set()
for text in data['message']:
    
    text = text.lower().replace('\n', ' ').replace('\r', '').strip()
    text = re.findall(r"[\u0E00-\u0E7Fa-zA-Z']+", text)
    text = ' '.join(text)

    word_tokens = THAI2FIT_TOKENIZER.word_tokenize(text)
    filtered_sentence = set([w for w in word_tokens])

    word_list.update(filtered_sentence)

In [ ]:
len(word_list)

### Clean Empty Data On Merge

In [ ]:
data = pd.read_csv("./data/_csv/3hnum3mum_message_1.csv")

In [ ]:
data = data.dropna()

In [ ]:
def clean_data(data):
    #filter na
    data = data.dropna()
    # filter common chat
    # data = data.drop(data.index[ data['category'] == 'C' ])
    # sort data by category
    data = data.sort_values(by=['category'])

    data = data.reset_index()
    data = data[['message', 'category']]

    return data

In [ ]:
data = clean_data(data)

### SemiAuto Label

In [2]:
from project_module.message_classifier import MessageClassifier

In [3]:
message_classifier = MessageClassifier()
message_classifier.load_model(
    model_path='./models/mlp_model.joblib',
    json_classes_path='./classes.json',
    scaler_path='./models/scaler.joblib',
)

In [4]:
message_data_converter = MessageDataConverter()
message_data_converter.prelabel_csv('./data/csv/cintania53_message_1.csv', prelabel_model= message_classifier)